<a href="https://colab.research.google.com/github/ialara/actf/blob/e2e-prototype/absorption_e2e_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
from collections import Counter
rng = np.random.default_rng()

In [116]:
class Pilot:
    ftu_sorties = 59
    def __init__(self, i, f16_sorties=59, tos=0, experienced=False):
        self.id = i
        self.f16_sorties = f16_sorties
        self.tos = tos
        self.experienced = experienced
        self.arrived_month = 0
        self.in_ug = False
        self.ug = None
        self.ride_num = None
        self.quals = []
        self.tte = 0

    def set_arrived_month(self, month):
        self.arrived_month = month

    def increment_tos(self, months=1):
        self.tos += months

    def increment_f16_sorties(self, sortie_increment=1):
        self.f16_sorties += sortie_increment
        if self.in_ug:
          self.ride_num += sortie_increment

    def enroll_in_ug(self, ug):
        self.in_ug = True
        self.ug = ug
        self.ride_num = 0
      
    def disenroll_from_ug(self):
        self.in_ug = False
        self.ug = None
        self.ride_num = None

    def award_qual(self, qual):
        if qual not in self.quals:
            self.quals.append(qual)

    def print_(self):
        msg = [f'ID: {self.id:2d} | EXP: {"Y" if self.experienced else "N"} | ',
               f'TOS: {self.tos:2d} | STY: {self.f16_sorties:3d} | ARR: {self.arrived_month:2d} | ',
               f'TTE: {self.tte if self.experienced else "- ":2} | QUAL: {self.quals}']
        if self.ug is not None:
            msg.append(f' | UG: {self.ug} #{self.ride_num}')

        print(*msg)

class Squadron:
    def __init__(self, name='default_Squadron'):
        self.name = name
        self.pilots = []
        self.pid = 0

    def assign_pilot(self, pilot, arrived_month=0):
        self.pilots.append(pilot)
        pilot.set_arrived_month(arrived_month)

    def set_monthly_sorties_available(self, capacity=240):
        self.monthly_sorties_available = capacity

    def _next_pid(self):
        self.pid += 1
        return self.pid

    def populate_initial(self, num_pilots=35, prop_EXP=0.5, prop_IP=0.4):
        num_EXP = rng.binomial(num_pilots, prop_EXP)
        num_IP = rng.binomial(num_EXP, prop_IP)
        num_INX = num_pilots - num_EXP

        # Parameters
        min_TOS_INX = 0
        max_TOS_INX = 24
        min_TOS_EXP = 16
        max_TOS_EXP = 32

        min_sorties_INX = Pilot.ftu_sorties
        max_sorties_INX = Syllabus.exp_sortie_rqmt
        min_sorties_EXP = Syllabus.exp_sortie_rqmt
        max_sorties_EXP = 2*Syllabus.exp_sortie_rqmt

        # Assign INX pilots
        for _ in range(num_INX):
            my_tos = rng.integers(min_TOS_INX, max_TOS_INX)
            my_sorties = rng.integers(min_sorties_INX, max_sorties_INX)
            my_sorties = min(my_sorties, min_sorties_INX + (my_tos+1)*5)
            inx_pilot = Pilot(self._next_pid(), f16_sorties=my_sorties, tos=my_tos)
            
            if inx_pilot.tos > 2: # Assume initial population completes MQT in specified timeline
                inx_pilot.increment_f16_sorties(9) # Duration of MQT
                inx_pilot.award_qual('WG')
            else:
                my_ug_prog = rng.integers(9) # MQT and FLUG have same duration
                if inx_pilot.f16_sorties < 75:
                    inx_pilot.enroll_in_ug('MQT')
                    inx_pilot.increment_f16_sorties(my_ug_prog)
                elif inx_pilot.f16_sorties < 200:
                    inx_pilot.award_qual('WG')
                    if rng.random() < 0.7:
                        inx_pilot.enroll_in_ug('FLUG')
                        inx_pilot.increment_f16_sorties(my_ug_prog)
                else:
                    inx_pilot.award_qual('WG')
                    if rng.random() < 0.7:
                        inx_pilot.award_qual('FL')
            self.assign_pilot(inx_pilot)
          
        # Assign EXP pilots
        ips_remaining = num_IP
        for _ in range(num_EXP):
            my_tos = rng.integers(min_TOS_EXP, max_TOS_EXP)
            my_sorties = rng.integers(min_sorties_EXP, max_sorties_EXP)
            exp_pilot = Pilot(self._next_pid(), f16_sorties=my_sorties, tos=my_tos,
                              experienced = True)
            exp_pilot.award_qual(Syllabus.exp_qual_rqmt)
            if ips_remaining > 0:
                exp_pilot.award_qual('IP')
                ips_remaining -= 1
            elif rng.random() < 0.2:
              exp_pilot.enroll_in_ug('IPUG')
              exp_pilot.increment_f16_sorties(rng.integers(9))

            self.assign_pilot(exp_pilot)

    def inflow_from_ftu(self, num_pilots=15, arrival_month = 0):
        for _ in range(num_pilots):
            ftu_sortie_delta = rng.integers(-5, 5)
            my_f16_sorties = Pilot.ftu_sorties + ftu_sortie_delta
            new_pilot = Pilot(self._next_pid(), f16_sorties = my_f16_sorties)
            new_pilot.set_arrived_month(arrival_month)
            self.assign_pilot(new_pilot)

    def inflow_nth_tour(self, num_pilots=20, prop_IP=0.4, prop_WG=0.1):
        num_IP = rng.binomial(num_pilots, prop_IP)
        num_WG = rng.binomial(num_pilots, prop_WG)
        num_FL = num_pilots - num_IP - num_WG

        # Parameters
        min_sorties_WG = int(0.7*Pilot.exp_sorties)
        max_sorties_WG = int(0.9*Pilot.exp_sorties)
        min_sorties_FL = int(0.9*Pilot.exp_sorties)
        max_sorties_FL = int(2*Pilot.exp_sorties)
        min_sorties_IP = int(1.5*Pilot.exp_sorties)
        max_sorties_IP = 4*Pilot.exp_sorties

    def inflow_pilots(self, num_pilots=15, arrival_month=0):

        for _ in range(num_pilots):
            self.assign_pilot(Pilot(self._next_pid()), arrival_month)

    def outflow_pilots(self, tos_threshold=32):
        removed_pilots = [p for p in self.pilots if p.tos >= tos_threshold]
        self.pilots = [p for p in self.pilots if p not in set(removed_pilots)]
        return removed_pilots

    def fly_month(self, sorties_available=None, INX_sortie_pct=0.6):
        if sorties_available is None:
            sorties_available = self.monthly_sorties_available
        INX_sorties = int(sorties_available * INX_sortie_pct)
        INX_pilots = [p for p in self.pilots if not p.experienced]
        num_INX_pilots = len(INX_pilots)

        rng.shuffle(INX_pilots)
        INX_sorties_remaining = INX_sorties
        for p in INX_pilots:
            my_draw = rng.binomial(INX_sorties, 1 / num_INX_pilots)
            my_SCM = min(my_draw, INX_sorties_remaining)
            print(f'PID {p.id:2d}: drew {my_draw:2d}, flying {my_SCM:2d}.')
            p.scm = my_SCM
            p.increment_f16_sorties(p.scm)
            if p.in_ug:
                p.ride_num += p.scm            
            INX_sorties_remaining -= p.scm

        print(f'EOM INX sorties remaining: {INX_sorties_remaining}')

    def update_qualifications(self, sim_month, ugs):
        for p in self.pilots:
            was_experienced = p.experienced
            p.experienced = Syllabus.meets_EXP_criteria(p.f16_sorties, p.quals)
            if p.experienced and not was_experienced: # "became" EXP
                # Calculate TTE
                p.tte = sim_month - p.arrived_month
                print(f'PID {p.id} experienced. TTE: {p.tte} months')

            if p.in_ug and p.ride_num + p.scm >= ugs[p.ug].duration:
                # Upgrade complete, award qualification
                print(f'PID {p.id} completed {p.ug}. Awarded {ugs[p.ug].award}.')
                p.award_qual(ugs[p.ug].award)
                p.disenroll_from_ug()            

    def age_squadron(self, months=1):
        for p in self.pilots:
            p.increment_tos(months)

    def summarize(self):
        quals = Counter(self._get_highest_quals())
        ugs = Counter(self._get_ug_enrollment())
        exp = Counter(self._get_experience())

        exp_str = [f'{"EXP" if q else "INX":6s} -> {num:2d} | ' for q, num in exp.items()]
        print('EXPR:', *exp_str, f'PILOTS -> {exp[True] + exp[False]} | {exp[True]/(exp[True]+exp[False])*100:.0f}% EXP')
        quals_str = [f'{q if q is not "" else "(none)":6s} -> {num:2d} | ' for q, num in quals.items()]
        print('QUAL:', *quals_str)
        ugs_str = [f'{q if q is not None else "(none)":6s} -> {num:2d} | ' for q, num in ugs.items()]
        print('UPGS:', *ugs_str)

    def _get_highest_quals(self):
        return [p.quals[-1] if len(p.quals) > 0 else '' for p in self.pilots]

    def _get_ug_enrollment(self):
        return [p.ug for p in self.pilots]

    def _get_experience(self):
        return [p.experienced for p in self.pilots]

    def print_(self):
        for p in self.pilots:
            p.print_()

class Simulation:
    def __init__(self, run_num=0):
        self.run_num = run_num
        self.month_num = 0

    def setup(self, initial_size=35, monthly_sortie_capacity=240):
        self.sq = Squadron('test_Squadron')
        self.sq.populate_initial(initial_size)
        self.sq.set_monthly_sorties_available(monthly_sortie_capacity)
        self.syllabi = {s.name: s for s in [Syllabus('MQT', 9, 'WG'),
                                            Syllabus('FLUG', 9, 'FL'),
                                            Syllabus('IPUG', 9, 'IP')]}

    def step_month(self, num_months=1, inflow_size=5, tos_threshold=3):
        self.month_num += 1
        print(f'---SIM MONTH {self.month_num}---')
        self.sq.inflow_pilots(inflow_size, self.month_num)
        print('After inflow:')
        self.sq.summarize()
        self.sq.fly_month()
        # print('After flying:')
        # self.sq.summarize()
        self.sq.age_squadron()
        # print('After aging:')
        # self.sq.summarize()
        self.sq.update_qualifications(self.month_num, self.syllabi)
        self.sq.outflow_pilots(tos_threshold)
        print('After outflow:')
        self.sq.summarize()

class Syllabus:
    exp_sortie_rqmt = 250
    exp_qual_rqmt = 'FL'
    def __init__(self, name, duration, award):
        self.name = name
        self.duration = duration
        self.award = award   
    @classmethod
    def meets_EXP_criteria(pilot_sorties, pilot_qualifications):
        return (pilot_sorties >= Syllabus.exp_sortie_rqmt and 
               Syllabus.exp_qual_rqmt in pilot_qualifications)

In [117]:
sq = Squadron('test_Sq')
sq.populate_initial(30)
sq.summarize()

EXPR: INX    -> 16 |  EXP    -> 14 |  PILOTS -> 30 | 47% EXP
QUAL: WG     -> 14 |  (none) ->  2 |  IP     ->  3 |  FL     -> 11 | 
UPGS: (none) -> 27 |  MQT    ->  2 |  IPUG   ->  1 | 


In [118]:
sq.print_()

ID:  1 | EXP: N |  TOS:  9 | STY: 118 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  2 | EXP: N |  TOS: 15 | STY: 148 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  3 | EXP: N |  TOS: 21 | STY: 157 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  4 | EXP: N |  TOS: 11 | STY: 128 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  5 | EXP: N |  TOS: 10 | STY: 105 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  6 | EXP: N |  TOS: 20 | STY: 138 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  7 | EXP: N |  TOS: 18 | STY: 163 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  8 | EXP: N |  TOS: 23 | STY: 188 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  9 | EXP: N |  TOS:  0 | STY:  69 | ARR:  0 |  TTE: -  | QUAL: []  | UG: MQT #8
ID: 10 | EXP: N |  TOS:  2 | STY:  76 | ARR:  0 |  TTE: -  | QUAL: []  | UG: MQT #2
ID: 11 | EXP: N |  TOS:  9 | STY: 118 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID: 12 | EXP: N |  TOS: 20 | STY: 101 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID: 13 | EXP: N |  TOS: 18 | STY: 163 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID: 14 | EXP: N | 

In [ ]:
sim = Simulation()
sim.setup(5)
sim.sq.summarize()

ID:  1 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  2 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  3 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  4 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  5 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []


In [ ]:
sim.sq.pilots[-2].enroll_in_ug('MQT')
sim.sq.pilots[-1].award_qual('FL')
sim.sq.pilots[-1].f16_sorties = 249

In [ ]:
for _ in range(5):
    sim.step_month()

---SIM MONTH 1---
After inflow:
ID:  1 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  2 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  3 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []
ID:  4 | EXP: N |  TOS:  0 | STY:  59 | ARR:  0 |  TTE: -  | QUAL: []  | UG: MQT #0
ID:  5 | EXP: N |  TOS:  0 | STY: 249 | ARR:  0 |  TTE: -  | QUAL: ['FL']
ID:  6 | EXP: N |  TOS:  0 | STY:  59 | ARR:  1 |  TTE: -  | QUAL: []
ID:  7 | EXP: N |  TOS:  0 | STY:  59 | ARR:  1 |  TTE: -  | QUAL: []
ID:  8 | EXP: N |  TOS:  0 | STY:  59 | ARR:  1 |  TTE: -  | QUAL: []
ID:  9 | EXP: N |  TOS:  0 | STY:  59 | ARR:  1 |  TTE: -  | QUAL: []
ID: 10 | EXP: N |  TOS:  0 | STY:  59 | ARR:  1 |  TTE: -  | QUAL: []
PID  9: drew 16, flying 16.
PID  4: drew  9, flying  9.
PID  7: drew 12, flying 12.
PID  1: drew 13, flying 13.
PID  3: drew 19, flying 19.
PID  5: drew 16, flying 16.
PID  6: drew 15, flying 15.
PID  8: drew 14, flying 14.
PID  2: drew 18, flying 18

In [ ]:
sim.sq.pilots[-1].f16_sorties=249
sim.sq.pilots[-1].award_qual('FL')
sim.sq.summarize()

ID:  1 | EXP: N |  TOS:  1 | STY:  70 | ARR:  0 |  TTE: -  | QUAL: []
ID:  2 | EXP: N |  TOS:  1 | STY:  73 | ARR:  0 |  TTE: -  | QUAL: []
ID:  3 | EXP: N |  TOS:  1 | STY:  71 | ARR:  0 |  TTE: -  | QUAL: []
ID:  4 | EXP: N |  TOS:  1 | STY:  79 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  5 | EXP: N |  TOS:  1 | STY:  71 | ARR:  0 |  TTE: -  | QUAL: []
ID:  6 | EXP: N |  TOS:  1 | STY:  80 | ARR:  1 |  TTE: -  | QUAL: []
ID:  7 | EXP: N |  TOS:  1 | STY:  74 | ARR:  1 |  TTE: -  | QUAL: []
ID:  8 | EXP: N |  TOS:  1 | STY:  76 | ARR:  1 |  TTE: -  | QUAL: []
ID:  9 | EXP: N |  TOS:  1 | STY:  72 | ARR:  1 |  TTE: -  | QUAL: []
ID: 10 | EXP: N |  TOS:  1 | STY: 249 | ARR:  1 |  TTE: -  | QUAL: ['FL']


In [ ]:
sim.step_month()

---SIM MONTH 2---
After inflow:
ID:  1 | EXP: N |  TOS:  1 | STY:  70 | ARR:  0 |  TTE: -  | QUAL: []
ID:  2 | EXP: N |  TOS:  1 | STY:  73 | ARR:  0 |  TTE: -  | QUAL: []
ID:  3 | EXP: N |  TOS:  1 | STY:  71 | ARR:  0 |  TTE: -  | QUAL: []
ID:  4 | EXP: N |  TOS:  1 | STY:  79 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  5 | EXP: N |  TOS:  1 | STY:  71 | ARR:  0 |  TTE: -  | QUAL: []
ID:  6 | EXP: N |  TOS:  1 | STY:  80 | ARR:  1 |  TTE: -  | QUAL: []
ID:  7 | EXP: N |  TOS:  1 | STY:  74 | ARR:  1 |  TTE: -  | QUAL: []
ID:  8 | EXP: N |  TOS:  1 | STY:  76 | ARR:  1 |  TTE: -  | QUAL: []
ID:  9 | EXP: N |  TOS:  1 | STY:  72 | ARR:  1 |  TTE: -  | QUAL: []
ID: 10 | EXP: N |  TOS:  1 | STY: 249 | ARR:  1 |  TTE: -  | QUAL: ['FL']
ID: 11 | EXP: N |  TOS:  0 | STY:  59 | ARR:  2 |  TTE: -  | QUAL: []
ID: 12 | EXP: N |  TOS:  0 | STY:  59 | ARR:  2 |  TTE: -  | QUAL: []
ID: 13 | EXP: N |  TOS:  0 | STY:  59 | ARR:  2 |  TTE: -  | QUAL: []
ID: 14 | EXP: N |  TOS:  0 | STY:  59 | ARR:  2 | 

In [ ]:
sim.step_month()

---SIM MONTH 3---
After inflow:
ID:  1 | EXP: N |  TOS:  2 | STY:  79 | ARR:  0 |  TTE: -  | QUAL: []
ID:  2 | EXP: N |  TOS:  2 | STY:  78 | ARR:  0 |  TTE: -  | QUAL: []
ID:  3 | EXP: N |  TOS:  2 | STY:  85 | ARR:  0 |  TTE: -  | QUAL: []
ID:  4 | EXP: N |  TOS:  2 | STY:  90 | ARR:  0 |  TTE: -  | QUAL: ['WG']
ID:  5 | EXP: N |  TOS:  2 | STY:  83 | ARR:  0 |  TTE: -  | QUAL: []
ID:  6 | EXP: N |  TOS:  2 | STY:  90 | ARR:  1 |  TTE: -  | QUAL: []
ID:  7 | EXP: N |  TOS:  2 | STY:  78 | ARR:  1 |  TTE: -  | QUAL: []
ID:  8 | EXP: N |  TOS:  2 | STY:  83 | ARR:  1 |  TTE: -  | QUAL: []
ID:  9 | EXP: N |  TOS:  2 | STY:  81 | ARR:  1 |  TTE: -  | QUAL: []
ID: 10 | EXP: Y |  TOS:  2 | STY: 263 | ARR:  1 |  TTE:  1 | QUAL: ['FL']
ID: 11 | EXP: N |  TOS:  1 | STY:  72 | ARR:  2 |  TTE: -  | QUAL: []
ID: 12 | EXP: N |  TOS:  1 | STY:  73 | ARR:  2 |  TTE: -  | QUAL: []
ID: 13 | EXP: N |  TOS:  1 | STY:  59 | ARR:  2 |  TTE: -  | QUAL: []
ID: 14 | EXP: N |  TOS:  1 | STY:  69 | ARR:  2 | 